In [4]:
from music21 import *
import numpy as np
import glob
import math
import os
import random
from cntk import Trainer, Axis
from cntk.learner import adam_sgd, momentum_sgd, momentum_as_time_constant_schedule, learning_rate_schedule, UnitType
from cntk.ops import input_variable, classification_error, squared_error
from cntk.persist import load_model, save_model
from cntk.blocks import LSTM, Stabilizer
from cntk.layers import Recurrence, Dense, Dropout, BatchNormalization
from cntk.utils import get_train_eval_criterion, get_train_loss
from cntk.device import set_default_device, gpu

path = "PianoMidiFiles3"

In [113]:
lowerBound = 24 # = [python] index 0 ([R] 1. possision)
upperBound = 102 # = first index out of bound => len =78
nNotes = upperBound-lowerBound

def sample(z,prime_data=np.array([[0]*((upperBound-lowerBound)*2+4+12)],dtype=np.float32),length=300):#,threshold=1.11):
    output=np.zeros((length, 173), dtype=np.float32)
    output[:,172]=[l for l in range(length)]
    p=0
    outlag=[0]*156
    for l in range(length):
        output[l,156:160]=time2Beat(l)[0]
    if len(prime_data[0])!=172:
        prime_data[:,0:172]
    randomNumb=np.random.choice(range(78))
    if len(prime_data)==1: 
        prime_data[0,randomNumb*2] = 1
        prime_data[0,randomNumb*2+1] = 1
        prime_data[0,160:172]=numNotes(prime_data[0,:])
    for i in range(len(prime_data)):
        x=prime_data[i]
        inputen=[np.array([x],dtype=np.float32)]
        # inputen=[np.array([data],dtype=np.float32)]
        # inputen=[np.array([x/np.maximum(x.sum(),1)],dtype=np.float32)]
        if i==0:
            arguments=(inputen,[True])
        else:
            p = z.eval(arguments)
            arguments=(inputen,[False])
        outlag=inputen[0][0][0:156]
    #x=prime_data[len(prime_data)-1]
    for i in range(length):
        p = z.eval(arguments)        
        output[i,0:156] = Output2data(p,outlag)# ,threshold=threshold)
        outlag = output[i,0:156]
        output[i,160:172]=numNotes(output[i,:])
        x=output[i,0:172]
        inputen=[np.array([x],dtype=np.float32)]
        # inputen=[np.array([x/np.maximum(x.sum(),1)],dtype=np.float32)]
        arguments=(inputen,[False])
    return(output)#return([list(testout[i]) for i in range(300)])

def numNotes(notes):
    for inote in range(78):
        notes[160+inote%12]+=1/4*notes[inote*2+1]
    notes[160:172]=np.minimum.reduce([notes[160:172],[1]*12])#[min(notes[k],1) for k in range(159,172)]
    return(notes[160:172])

def Output2data(p,outlag):
    p=p[0][0]
    out=(p>np.random.random_sample((1, len(p))))*1 #np.array([[0.6]*len(p)]))#
    for i in range(78):
        if outlag[i*2+1]==0 and out[0][i*2]==0:
            out[0][i*2+1]=0
#        elif out[0][i*2]==1:
#            out[0][i*2+1]=1
        elif out[0][i*2]==1 and out[0][i*2+1]==0:
            out[0][i*2+1]=(random.uniform(0, 1)<0.65)*1
            out[0][i*2]=out[0][i*2+1]
    return(out)
    # return(list((p>p.mean()+threshold)*1))
    
def data2MusicMatric(data):
    timeIndex=0
    MusicMatrix=[0]*len(data)
    MusicMatrixRow=[0]*(upperBound-lowerBound)
    for i in range(len(data)):
        for j in range(upperBound-lowerBound):
            MusicMatrixRow[j] = [int(data[i,j*2]),int(data[i,j*2+1])]
        MusicMatrix[i] = MusicMatrixRow+[[timeIndex]]
        timeIndex+=0.25
    return(MusicMatrix)

def MusicMinMatrix2midi(MusicMinMatrixPart,S,bpm=100):
    #S = stream.Score()
    nTime=len(MusicMinMatrixPart)
    S.insert(0,tempo.MetronomeMark(number=bpm))
    p1 = stream.Part()
    p1.id = 'part1'
    nCons=0
    thisIndex=0
    mLen = len(MusicMinMatrixPart[0])
    for j in range(78):
        for i in range(nTime):
            if (nCons!=0) and (MusicMinMatrixPart[i][j] in ([0,0],[1,1])):
                n.duration.quarterLength = 0.25*nCons
                p1.insert(MusicMinMatrixPart[thisIndex][mLen-1][0],n)
                nCons=0
            if MusicMinMatrixPart[i][j]==[1,1]:
                n = note.Note()
                n.midi = lowerBound+j
                thisIndex = i
                nCons+=1
            elif (MusicMinMatrixPart[i][j]==[0,1]) and (nCons!=0):
                nCons+=1
        if (nCons != 0):
            n.duration.quarterLength = 0.25*nCons
            p1.insert(MusicMinMatrixPart[thisIndex][mLen-1][0],n)
            nCons=0
    S.insert(0,p1)
    
def time2Beat(time):
    beat=[[math.floor((time%16)/8),math.floor((time%8)/4),math.floor((time%4)/2),time%2]]
    return(beat)

In [68]:
z=load_model('0512composer_epoch300.dnn')

In [114]:
# make output
# set output length
length=2000
testout=0
testout=sample(z,prime_data=np.array([[0]*((upperBound-lowerBound)*2+4+12)],dtype=np.float32),length=length) #,threshold=5)
musicM=data2MusicMatric(testout)
S = stream.Score()
MusicMinMatrix2midi(musicM,S,65)

In [61]:
prime_data=np.array([[0]*((upperBound-lowerBound)*2+4+12)],dtype=np.float32)
x=prime_data[0]
inputen=[np.array([x],dtype=np.float32)]
arguments=(inputen,[True])
#outtest=
# z.eval(arguments)
#print(outtest[0][0])

In [ ]:
#play Track
sp=midi.realtime.StreamPlayer(S)
sp.play()

In [115]:
#S.write('midi', 'FolderName/FileName.mid')
# nr 7 0.65
# nr 6 0.75
# nr 5 1 
# nr 4 0.7
# nr 3 0.5
# nr 2 0
# nr 1 1
S.write('midi', 'MLmusicOut/TestS54E300nr11.mid')

'MLmusicOut/TestS54E300nr11.mid'

In [ ]:
prime_data=np.array([[0]*(10)],dtype=np.float32)
x=prime_data[0]
inputen=[np.array([x],dtype=np.float32)]
arguments=(inputen,[True])